<a href="https://colab.research.google.com/github/Whoami666/HSE_Lab/blob/main/clean_models/MLPClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 4.3 MB/s eta 0:00:00


In [4]:
%cd drive
%cd MyDrive
%cd shiz

/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/shiz


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [6]:
df = pd.read_excel('df_standard_cleaned_21feb.xlsx') 
df1d = pd.read_excel('df_standard_only1dfeatures.xlsx')

df = df.set_index('ID')
df1d = df1d.set_index('ID')
df.head(1)

,sex,tds,chroma_cens_avg,chroma_cens_std,chroma_cqt_avg,chroma_cqt_std,chroma_stft_avg,chroma_stft_std,melspectrogram_avg,melspectrogram_std,...,spectral_flatness_std,spectral_rolloff_avg,spectral_rolloff_std,tempogram_avg,tempogram_std,tonnetz_avg,tonnetz_std,zero_crossing_rate_avg,zero_crossing_rate_std,tds_binary
ID,,,,,,,,,,,,,,,,,,,,,
PD-003,1,0,0.685706,0.428088,0.425039,0.296394,0.339246,0.434412,0.327867,0.348849,...,0.31499,0.397736,0.254215,0.747885,0.719312,0.691479,0.329593,0.197304,0.281692,0


Models

In [7]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.inspection import permutation_importance

from sklearn.linear_model import LogisticRegression
from skopt import BayesSearchCV
from sklearn.svm import SVC

from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits

from sklearn.model_selection import GridSearchCV

In [8]:
def calculate_scores(model, givenx, giveny, xtest):
  kf = KFold(n_splits=10)
  crossval_f1 = cross_val_score(model, givenx, giveny, scoring='f1_macro', cv = kf, verbose=False)
  mean_score = crossval_f1.mean()
  std_score = crossval_f1.std()
  print('f1_macro', mean_score, '±', std_score)

  crossval_accuracy = cross_val_score(model, givenx, giveny, scoring='accuracy', cv = kf, verbose=False)
  mean_score = crossval_accuracy.mean()
  std_score = crossval_accuracy.std()
  print('accuracy', mean_score, '±', std_score)

  crossval_recall = cross_val_score(model, givenx, giveny, scoring='recall_macro', cv = kf, verbose=False)
  mean_score = crossval_recall.mean()
  std_score = crossval_recall.std()
  print('recall', mean_score, '±', std_score)

  if giveny.nunique() == 2:
    y_proba = model.predict_proba(xtest)
    print('roc_auc_score', roc_auc_score(y_test, y_proba[:,1]))

In [15]:
def mlp_class(xtrain, ytrain, xtest, ytest, xgiven, ygiven):
  sc_X = StandardScaler()
  X_trainscaled=sc_X.fit_transform(xtrain)
  X_testscaled=sc_X.transform(xtest)

  warnings.filterwarnings('ignore')
  opt = BayesSearchCV(
      MLPClassifier(),
      {
          'activation': ['identity', 'logistic', 'tanh', 'relu'],
          'random_state': [1, 2, 3],
          'solver': ['sgd', 'adam']
      },
      n_iter=32,
      scoring='f1_macro',
      cv=3
  )

  opt.fit(X_trainscaled, y_train)

  print("val. score: %s" % opt.best_score_)
  print("best_params: %s" % opt.best_params_, '\n')
  print("test score: %s" % opt.score(X_testscaled, y_test))

  params = opt.best_params_
  mlp = MLPClassifier(hidden_layer_sizes=(256,128,64,32), activation=params['activation'], random_state=params['random_state'], solver=params['solver']).fit(X_trainscaled, y_train)
  y_pred = mlp.predict(X_testscaled)
  print(classification_report(y_test,y_pred))

  calculate_scores(mlp, xgiven, ygiven, xtest)

**Non-binary**

All features

In [16]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [17]:
mlp_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.5648822766036337
best_params: OrderedDict([('activation', 'identity'), ('random_state', 1), ('solver', 'sgd')]) 

test score: 0.3149606299212599
              precision    recall  f1-score   support

           0       0.91      0.97      0.94        60
           1       0.25      0.17      0.20         6
           2       0.00      0.00      0.00         2

    accuracy                           0.87        68
   macro avg       0.39      0.38      0.38        68
weighted avg       0.82      0.87      0.84        68

f1_macro 0.7411054994388329 ± 0.3281093493975115
accuracy 0.8882352941176471 ± 0.19939354419621522
recall 0.7666666666666666 ± 0.2905932629027116


1d

In [18]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [19]:
mlp_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.5648822766036337
best_params: OrderedDict([('activation', 'identity'), ('random_state', 1), ('solver', 'sgd')]) 

test score: 0.3149606299212599
              precision    recall  f1-score   support

           0       0.91      0.97      0.94        60
           1       0.25      0.17      0.20         6
           2       0.00      0.00      0.00         2

    accuracy                           0.87        68
   macro avg       0.39      0.38      0.38        68
weighted avg       0.82      0.87      0.84        68

f1_macro 0.7411054994388329 ± 0.3281093493975115
accuracy 0.8882352941176471 ± 0.19939354419621522
recall 0.7666666666666666 ± 0.2905932629027116


**Binary**

All features

In [20]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds_binary

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [21]:
mlp_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.7021633384536611
best_params: OrderedDict([('activation', 'identity'), ('random_state', 1), ('solver', 'adam')]) 

test score: 0.662809917355372
              precision    recall  f1-score   support

           0       0.89      0.93      0.91        60
           1       0.20      0.12      0.15         8

    accuracy                           0.84        68
   macro avg       0.54      0.53      0.53        68
weighted avg       0.81      0.84      0.82        68

f1_macro 0.6183528766530714 ± 0.261078144379279
accuracy 0.8290441176470587 ± 0.17515787799349303
recall 0.6280724789915966 ± 0.25303322552682445
roc_auc_score 0.7041666666666666


1d

In [22]:
X = df1d.drop(['tds', 'tds_binary'], axis=1)
y = df1d.tds_binary

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [23]:
mlp_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.6010710718272855
best_params: OrderedDict([('activation', 'tanh'), ('random_state', 3), ('solver', 'adam')]) 

test score: 0.7527272727272727
              precision    recall  f1-score   support

           0       0.92      0.97      0.94        60
           1       0.60      0.38      0.46         8

    accuracy                           0.90        68
   macro avg       0.76      0.67      0.70        68
weighted avg       0.88      0.90      0.89        68

f1_macro 0.6085301944172912 ± 0.2633622993103361
accuracy 0.8702205882352942 ± 0.19102121806908037
recall 0.6409926470588235 ± 0.23535449880121018
roc_auc_score 0.18958333333333333
